In [30]:
%%prun -s cumulative 
%matplotlib inline
%run data_path.ipynb
import numpy as np
import time
import matplotlib.pyplot as plt
import pandas as pd
import os
from pympler import asizeof
import cylib.cydepth as cyd

start_time = time.time()
inputfile = umap_files["BBv2"]

class Ref_map:
    def __init__(self,refname,reflen,spos,epos,rlen):
        self.refname = refname
        self.reflen = reflen
        self.pos = [[spos,epos,rlen]]
        
    def __str__(self):
        return '{0} {1}'.format(self.refname,self.reflen)
        
def getrefpos(file):
    with open(file, 'r') as f:
        for line in f:
            llist = line.split()
            ref = llist[5].split('-')[1]
            reflen = int(llist[6])
            pos = [int(llist[7]), int(llist[8])]
            rlen = [int(llist[1])]
            if ref in refdict:
                refdict[ref].pos.append(pos+rlen)
            else:
                if ref.find("Saccharomyces") != -1: continue
                if ref.find("Cryptococcus") != -1: continue
                if ref.find("plasmid") != -1: continue
    #             if ref.find("Lactobacillus") == -1: continue
                refdict[ref] = Ref_map(ref,reflen,*pos,*rlen)

def deparr(obj, arr):
    for spos,epos,rlen in obj.pos:
        arr[0][spos:epos+1] += 1
        arr[1][spos:epos+1] += rlen


        
refdict={}
getrefpos(inputfile)

for ref,refobj in refdict.items():
#   if ref.find("Test_depth") == -1: continue
    print(refobj)
    refarr = np.zeros([2,refobj.reflen+1])
    looparr(refobj,refarr)
#     deparr(refobj,refarr)
    avgarr = np.floor_divide(refarr[1], refarr[0], out=np.zeros_like(refarr[1]), where=refarr[0]!=0)
#       find zero position
    zpos=[]
    zpos = cyd.cpfind_extreme(avgarr, avgarr.size, 0, 0)
    print("0 depth: ", zpos)
    exv = 1000
    big=[]
    big = cyd.cpfind_extreme(refarr[0], refarr[0].size, exv, 1)
    print(">", exv, "depth: ", big)
    ixs = np.arange(refobj.reflen+1)
    plt.figure(figsize=(15,6))
######### plot per base coverage    
    plt.subplot(1,2,1)
    plt.plot(ixs,refarr[0],label=ref)
    plt.legend(loc='upper right')
######### plot per base average length coverage
    plt.subplot(1,2,2)
    plt.plot(ixs,avgarr,label=ref)
    plt.legend(loc='upper right')
    plt.show()
    print('\n\n')

          

print("--- %s seconds ---" % (time.time() - start_time))    

Staphylococcus_aureus_chromosome 2718780


KeyboardInterrupt: 